Name: Dev Patel 

Course: DS4400 Data Mining and Machine Learning 1

Prof: Silvio Amir

University: Northeastern University

Problem 5: Gradient Descent for Linear Regression

1. Implement gradient descent for training linear regression.
2. Vary learning rate $\alpha \in \{0.01, 0.1, 0.5\}$ and report $\theta$ after 10, 50, and 100 iterations. Table of MSE and R² on train/test.
3. Observations on convergence behavior.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

exclude_cols = ['id', 'date', 'zipcode', 'price', '', 'Unnamed: 0']
feature_cols = [c for c in train_df.columns if c not in exclude_cols]

X_train = train_df[feature_cols].values.astype(float)
y_train = train_df['price'].values.astype(float)
X_test = test_df[feature_cols].values.astype(float)
y_test = test_df['price'].values.astype(float)

# Standardize features (zero mean, unit variance) for gradient descent stability
train_mean = X_train.mean(axis=0)
train_std = X_train.std(axis=0)
train_std[train_std == 0] = 1  # avoid division by zero

X_train_scaled = (X_train - train_mean) / train_std
X_test_scaled = (X_test - train_mean) / train_std

print(f"Features: {feature_cols}")
print(f"X_train: {X_train_scaled.shape}, y_train: {y_train.shape}")

Features: ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long', 'sqft_living15', 'sqft_lot15']
X_train: (1000, 17), y_train: (1000,)


In [3]:
def gradient_descent(X, y, alpha, n_iters):
    """
    Gradient descent for linear regression.
    
    Cost: J(θ) = (1/2n) * Σ(Xθ - y)²
    Gradient: ∇J = (1/n) * X^T (Xθ - y)
    Update: θ := θ - α * ∇J
    
    X: (n, p) feature matrix (no intercept column; added inside)
    Returns: theta (p+1,), list of theta snapshots at requested iterations
    """
    n = len(y)
    X_design = np.column_stack([np.ones(n), X])  # add intercept
    p = X_design.shape[1]
    theta = np.zeros(p)  # initialize at zero
    
    for i in range(1, n_iters + 1):
        residual = X_design @ theta - y          # (n,)
        gradient = (1 / n) * (X_design.T @ residual)  # (p,)
        theta = theta - alpha * gradient
    
    return theta

def predict_gd(X, theta):
    """Predict using theta. X should NOT include intercept."""
    X_design = np.column_stack([np.ones(len(X)), X])
    return X_design @ theta

In [4]:
alphas = [0.01, 0.1, 0.3, 0.5]
iterations = [10, 50, 100]

rows = []
for alpha in alphas:
    for n_iter in iterations:
        theta = gradient_descent(X_train_scaled, y_train, alpha, n_iter)
        
        y_train_pred = predict_gd(X_train_scaled, theta)
        y_test_pred = predict_gd(X_test_scaled, theta)
        
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        
        rows.append({
            'α': alpha,
            'Iterations': n_iter,
            'Train MSE': train_mse,
            'Train R²': train_r2,
            'Test MSE': test_mse,
            'Test R²': test_r2
        })

results_df = pd.DataFrame(rows)
fmt = results_df.copy()
fmt['Train MSE'] = fmt['Train MSE'].map('{:,.0f}'.format)
fmt['Test MSE'] = fmt['Test MSE'].map('{:,.0f}'.format)
fmt['Train R²'] = fmt['Train R²'].map('{:.4f}'.format)
fmt['Test R²'] = fmt['Test R²'].map('{:.4f}'.format)
print("Gradient Descent Results:\n")
fmt

Gradient Descent Results:



,α,Iterations,Train MSE,Train R²,Test MSE,Test R²
0,0.01,10,"294,798,733,591",-1.5604,"350,525,097,299",-1.1024
1,0.01,50,"138,295,915,198",-0.2011,"170,376,668,653",-0.0219
2,0.01,100,"70,118,986,603",0.3910,"97,486,244,759",0.4153
3,0.10,10,"66,499,315,474",0.4224,"93,559,294,995",0.4388
4,0.10,50,"31,578,978,073",0.7257,"58,012,316,715",0.6521
5,0.10,100,"31,497,692,326",0.7264,"57,725,185,719",0.6538
6,0.30,10,"31,923,127,584",0.7227,"58,544,326,400",0.6489
7,0.30,50,"31,487,744,695",0.7265,"57,656,471,652",0.6542
8,0.30,100,"31,486,174,819",0.7265,"57,629,970,173",0.6543
9,0.50,10,"611,829,862,759,887",-5312.9211,"685,023,079,342,110",-4107.6524


In [5]:
# Compare best GD result with closed-form (Problem 3)
def fit_closed_form(X, y):
    X_design = np.column_stack([np.ones(len(X)), X])
    beta, *_ = np.linalg.lstsq(X_design, y, rcond=None)
    return beta

beta_cf = fit_closed_form(X_train_scaled, y_train)
y_train_cf = np.column_stack([np.ones(len(X_train_scaled)), X_train_scaled]) @ beta_cf
y_test_cf = np.column_stack([np.ones(len(X_test_scaled)), X_test_scaled]) @ beta_cf

print("--- Closed-form (optimal) ---")
print(f"Train MSE: {mean_squared_error(y_train, y_train_cf):,.0f}  R²: {r2_score(y_train, y_train_cf):.4f}")
print(f"Test  MSE: {mean_squared_error(y_test, y_test_cf):,.0f}  R²: {r2_score(y_test, y_test_cf):.4f}")

--- Closed-form (optimal) ---
Train MSE: 31,486,167,776  R²: 0.7265
Test  MSE: 57,628,154,706  R²: 0.6544


**Observations:**

- **Small learning rate (α = 0.01):** Convergence is very slow. After 10 iterations the model has barely improved. MSE is still huge and R² is negative. By 100 iterations it has improved, but it still has not converged to the optimal solution (R² ≈ 0.39 vs the optimal 0.73). Many more iterations would be needed.

- **Moderate learning rate (α = 0.1):** Good balance between speed and stability. By 50 iterations the metrics are close to the closed-form optimum, and by 100 iterations they are nearly identical (R² ≈ 0.726).

- **Larger learning rate (α = 0.3):** Converges fastest among the stable rates. It reaches near-optimal MSE/R² in roughly 50 iterations. This shows that a good learning rate can dramatically reduce the number of iterations needed.

- **Too-large learning rate (α = 0.5):** The algorithm diverges and the MSE explodes to astronomical values. The step size overshoots the minimum, so each iteration makes the parameters worse. This shows that the learning rate must be small enough relative to the curvature of the loss surface.

- **Convergence to the optimal solution:** With a suitable learning rate and enough iterations, gradient descent converges to the same solution as the closed-form method (Problem 3). For example, α = 0.3 at 100 iterations is nearly identical to the closed-form MSE/R².

- **Feature standardization is critical:** Without scaling, features have very different magnitudes (e.g., `sqft_lot` ~ 10,000 vs `waterfront` ~ 0/1), which creates an elongated loss surface. A single learning rate cannot work well for all features simultaneously, leading to either divergence or very slow convergence.